In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def dfdx(x,f):
    #d2y/dx2 = -y
    #define :
    
    #y = f[0]
    #dy/dx = z
    #z = f[1]
    
    y = f[0]
    z = f[1]
    
    #return derivatives
    dydx = np.zeros_like(f)
    dydx[0] = z
    dydx[1]= -1*y
    
    return dydx

In [3]:
def cash_karp_core_mv(x_i,y_i,nv,h,f):
    
    #cash karp is defined in terms of weighting varaibles
    ni  = 7
    nj  = 6
    ci  = np.zeros(ni)
    aij = np.zeros((ni,nj))
    bi  = np.zeros(ni)
    bis = np.zeros(ni)
    
    #input values for ci,aij,bi,bis
    ci[2] = 1./5.
    ci[3] = 3./10.
    ci[4] = 3./5.
    ci[5] = 1.
    ci[6] = 7./8.
    
    #j = 1
    aij[2,1] = 1./5
    aij[3,1] = 3./40.
    aij[4,1] = 3./10.
    aij[5,1] = -11./54.
    aij[6,1] = 1631./55296.
    
    #j = 2
    aij[3,2] = 9./40.
    aij[4,2] =-9./10.
    aij[5,2] = 5./2.
    aij[6,2] = 175./512.
    
    #j = 3  
    aij[4,3] = 6./5.
    aij[5,3] = -70./27.
    aij[6,3] = 575./13824.
    
    #j = 4
    aij[5,4] = 35./27.
    aij[6,4] = 44275./110592.
    
    #j = 5
    aij[6,5] = 253./4096.
    
    #bi
    bi[1] = 37./378.
    bi[2] = 0. 
    bi[3] = 250./621.
    bi[4] = 125./594.
    bi[5] = 0.0
    bi[6] = 512./1771.
    
    #bis
    bis[1] = 2825./27648.
    bis[2] = 0.0
    bis[3] = 18575./48384.
    bis[4] = 13525./55296.
    bis[5] = 277./14336.
    bis[6] = 1./4.
    
    #define the k array
    ki = np.zeros((ni,nv))
    
    #compute ki
    for i in range(1,ni):
        #computer xn+1 for i
        xn = x_i + ci[i]*h
        
        #compute temp y
        yn = y_i.copy()
        for j in range(1,i):
            yn += aij[i,j]*ki[j,:]

        #get k
        ki[i,:] = h*f(xn,yn)
        
        #get ynpo, ynpo*
        ynpo  = y_i.copy() 
        ynpos = y_i.copy()
        
        #print("")
        for i in range(1,ni):
            ynpo  += bi[i] *ki[i,:]
            ynpos += bis[i]*ki[i,:]
            #print(i,ynpo[0],ynpos[0])
            #print(i,ynpo[0],ynpos[0])         
            
        #get error
        Delta = np.fabs(ynpo-ynpos)
        
        #print("INSIDE Delta",Delta,ki)
        
        return ynpo, Delta

In [4]:
#Not my work, just the solution
def cash_karp_mv_ad(dfdx,x_i,y_i,nv,h,tol):
    
    #define a safetly scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterations
    imax = 1000
    
    #set an iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    #remember the step
    h_step = h
    
    #adjust the step
    while(Delta.max()/tol>1.0):
        
        #get out new y and error estimate
        y_ipo, Delta = cash_karp_core_mv(x_i,y_i,nv,h_step,dfdx)
        
        #if the error is too large, take a smaller step
        if(Delta.max()/tol>1.0):
            #our error is too large, decrease step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check iteration
        if(i>=imax):
            print("Too many iterations in cash_karp_mv_ad()")
            raise StopIteration("Ending after i = ",i)
            
        #iterate
        i += 1
        
    #next time, try a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step * H_NEW_FAC)
    
    
    #return the answer and step info
    return y_ipo, h_new, h_step
            

In [5]:
def cash_karp_mv(dfdx,a,b,y_a,tol,verbose=False):
    
    #dfdx is the derivative wrt x
    #a is the lower bound
    #b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define out starting step
    xi = a
    yi = y_a.copy()
    
    #an inital step size == make very small
    h = 1.0e-4 * (b-a)
    
    #sey a maximum number of iterations 
    imax = 10000
    
    #set an interation variable
    i = 0
    
    #set the number of couples odes to the size of y_a
    nv = len(y_a)
    
    #set the inital conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag = True
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        y_ipo, h_new, h_step = cash_karp_mv_ad(dfdx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = cash_karp_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #break 
            flag = False
            
        #update values
        xi += h_step
        yi = y_ipo.copy()
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_ipo = np.zeros((len(x),nv))
        y_ipo[0:len(x)-1,:] = y[:]
        y_ipo[-1,:] = yi[:]
        del y
        y = y_ipo
        
        #prevent too many iterations
        if(i>=imax):
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
            
        #iterate 
        i += 1
        
        #output some information
        if(verbose):
            s = "i = %3d\tx = %9.8f\ty = %9.8f\th = %9.8f\tb = %9.8f" % (i,xi,yi[0], h_step, b)
            print(s)
        
        #break if new xi is == b
        if(xi==b):
            flag = 0
            
    #return the answer
    return x,y

In [6]:
a = 0.0
b = 2.0 * np.pi
y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6
x,y = cash_karp_mv(dfdx,a,b,y_0,tolerance,verbose=True)
plt.plot(x,y[:,0],'o', label='y(x)')
plt.plot(x,y[:,1],'o', label='z(x)')
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlim([0,2*np.pi])
plt.ylim([-1.2,1.2])
plt.legend(frameon=False)


i =   1	x = 0.00021372	y = 0.00002092	h = 0.00021372	b = 6.28318531
i =   2	x = 0.00044462	y = 0.00004352	h = 0.00023090	b = 6.28318531
i =   3	x = 0.00067732	y = 0.00006630	h = 0.00023270	b = 6.28318531
i =   4	x = 0.00091019	y = 0.00008909	h = 0.00023288	b = 6.28318531
i =   5	x = 0.00114309	y = 0.00011189	h = 0.00023289	b = 6.28318531
i =   6	x = 0.00137598	y = 0.00013469	h = 0.00023290	b = 6.28318531
i =   7	x = 0.00160888	y = 0.00015748	h = 0.00023290	b = 6.28318531
i =   8	x = 0.00184177	y = 0.00018028	h = 0.00023290	b = 6.28318531
i =   9	x = 0.00207467	y = 0.00020308	h = 0.00023290	b = 6.28318531
i =  10	x = 0.00230756	y = 0.00022587	h = 0.00023290	b = 6.28318531
i =  11	x = 0.00254046	y = 0.00024867	h = 0.00023290	b = 6.28318531
i =  12	x = 0.00277335	y = 0.00027147	h = 0.00023290	b = 6.28318531
i =  13	x = 0.00300625	y = 0.00029426	h = 0.00023290	b = 6.28318531
i =  14	x = 0.00323915	y = 0.00031706	h = 0.00023290	b = 6.28318531
i =  15	x = 0.00347204	y = 0.00033986	h = 0.0002

i = 389	x = 0.09057605	y = 0.00886579	h = 0.00023290	b = 6.28318531
i = 390	x = 0.09080896	y = 0.00888859	h = 0.00023290	b = 6.28318531
i = 391	x = 0.09104186	y = 0.00891139	h = 0.00023290	b = 6.28318531
i = 392	x = 0.09127477	y = 0.00893418	h = 0.00023290	b = 6.28318531
i = 393	x = 0.09150767	y = 0.00895698	h = 0.00023290	b = 6.28318531
i = 394	x = 0.09174058	y = 0.00897978	h = 0.00023290	b = 6.28318531
i = 395	x = 0.09197348	y = 0.00900257	h = 0.00023290	b = 6.28318531
i = 396	x = 0.09220638	y = 0.00902537	h = 0.00023290	b = 6.28318531
i = 397	x = 0.09243929	y = 0.00904817	h = 0.00023290	b = 6.28318531
i = 398	x = 0.09267219	y = 0.00907096	h = 0.00023290	b = 6.28318531
i = 399	x = 0.09290510	y = 0.00909376	h = 0.00023290	b = 6.28318531
i = 400	x = 0.09313800	y = 0.00911656	h = 0.00023290	b = 6.28318531
i = 401	x = 0.09337091	y = 0.00913935	h = 0.00023290	b = 6.28318531
i = 402	x = 0.09360381	y = 0.00916215	h = 0.00023290	b = 6.28318531
i = 403	x = 0.09383672	y = 0.00918495	h = 0.0002

i = 688	x = 0.16021709	y = 0.01568199	h = 0.00023292	b = 6.28318531
i = 689	x = 0.16045001	y = 0.01570478	h = 0.00023292	b = 6.28318531
i = 690	x = 0.16068294	y = 0.01572758	h = 0.00023292	b = 6.28318531
i = 691	x = 0.16091586	y = 0.01575037	h = 0.00023292	b = 6.28318531
i = 692	x = 0.16114879	y = 0.01577317	h = 0.00023292	b = 6.28318531
i = 693	x = 0.16138171	y = 0.01579597	h = 0.00023292	b = 6.28318531
i = 694	x = 0.16161463	y = 0.01581876	h = 0.00023292	b = 6.28318531
i = 695	x = 0.16184756	y = 0.01584156	h = 0.00023292	b = 6.28318531
i = 696	x = 0.16208048	y = 0.01586436	h = 0.00023292	b = 6.28318531
i = 697	x = 0.16231341	y = 0.01588715	h = 0.00023292	b = 6.28318531
i = 698	x = 0.16254633	y = 0.01590995	h = 0.00023292	b = 6.28318531
i = 699	x = 0.16277926	y = 0.01593275	h = 0.00023292	b = 6.28318531
i = 700	x = 0.16301218	y = 0.01595554	h = 0.00023292	b = 6.28318531
i = 701	x = 0.16324511	y = 0.01597834	h = 0.00023292	b = 6.28318531
i = 702	x = 0.16347803	y = 0.01600114	h = 0.0002

i = 1061	x = 0.24710448	y = 0.02418512	h = 0.00023296	b = 6.28318531
i = 1062	x = 0.24733744	y = 0.02420792	h = 0.00023296	b = 6.28318531
i = 1063	x = 0.24757041	y = 0.02423072	h = 0.00023296	b = 6.28318531
i = 1064	x = 0.24780337	y = 0.02425351	h = 0.00023296	b = 6.28318531
i = 1065	x = 0.24803634	y = 0.02427631	h = 0.00023296	b = 6.28318531
i = 1066	x = 0.24826930	y = 0.02429911	h = 0.00023296	b = 6.28318531
i = 1067	x = 0.24850226	y = 0.02432190	h = 0.00023296	b = 6.28318531
i = 1068	x = 0.24873523	y = 0.02434470	h = 0.00023296	b = 6.28318531
i = 1069	x = 0.24896819	y = 0.02436750	h = 0.00023296	b = 6.28318531
i = 1070	x = 0.24920116	y = 0.02439029	h = 0.00023296	b = 6.28318531
i = 1071	x = 0.24943412	y = 0.02441309	h = 0.00023296	b = 6.28318531
i = 1072	x = 0.24966708	y = 0.02443589	h = 0.00023296	b = 6.28318531
i = 1073	x = 0.24990005	y = 0.02445868	h = 0.00023296	b = 6.28318531
i = 1074	x = 0.25013301	y = 0.02448148	h = 0.00023296	b = 6.28318531
i = 1075	x = 0.25036598	y = 0.0245

i = 1383	x = 0.32212595	y = 0.03152563	h = 0.00023301	b = 6.28318531
i = 1384	x = 0.32235896	y = 0.03154843	h = 0.00023301	b = 6.28318531
i = 1385	x = 0.32259197	y = 0.03157123	h = 0.00023301	b = 6.28318531
i = 1386	x = 0.32282498	y = 0.03159402	h = 0.00023301	b = 6.28318531
i = 1387	x = 0.32305799	y = 0.03161682	h = 0.00023301	b = 6.28318531
i = 1388	x = 0.32329101	y = 0.03163962	h = 0.00023301	b = 6.28318531
i = 1389	x = 0.32352402	y = 0.03166241	h = 0.00023301	b = 6.28318531
i = 1390	x = 0.32375703	y = 0.03168521	h = 0.00023301	b = 6.28318531
i = 1391	x = 0.32399004	y = 0.03170801	h = 0.00023301	b = 6.28318531
i = 1392	x = 0.32422305	y = 0.03173080	h = 0.00023301	b = 6.28318531
i = 1393	x = 0.32445607	y = 0.03175360	h = 0.00023301	b = 6.28318531
i = 1394	x = 0.32468908	y = 0.03177640	h = 0.00023301	b = 6.28318531
i = 1395	x = 0.32492209	y = 0.03179919	h = 0.00023301	b = 6.28318531
i = 1396	x = 0.32515510	y = 0.03182199	h = 0.00023301	b = 6.28318531
i = 1397	x = 0.32538812	y = 0.0318

...

i = 9271	x = 2.17554526	y = 0.21134489	h = 0.00023827	b = 6.28318531
i = 9272	x = 2.17578354	y = 0.21136769	h = 0.00023828	b = 6.28318531
i = 9273	x = 2.17602182	y = 0.21139048	h = 0.00023828	b = 6.28318531
i = 9274	x = 2.17626009	y = 0.21141328	h = 0.00023828	b = 6.28318531
i = 9275	x = 2.17649837	y = 0.21143608	h = 0.00023828	b = 6.28318531
i = 9276	x = 2.17673665	y = 0.21145887	h = 0.00023828	b = 6.28318531
i = 9277	x = 2.17697493	y = 0.21148167	h = 0.00023828	b = 6.28318531
i = 9278	x = 2.17721322	y = 0.21150447	h = 0.00023828	b = 6.28318531
i = 9279	x = 2.17745150	y = 0.21152726	h = 0.00023828	b = 6.28318531
i = 9280	x = 2.17768979	y = 0.21155006	h = 0.00023829	b = 6.28318531
i = 9281	x = 2.17792807	y = 0.21157285	h = 0.00023829	b = 6.28318531
i = 9282	x = 2.17816636	y = 0.21159565	h = 0.00023829	b = 6.28318531
i = 9283	x = 2.17840465	y = 0.21161845	h = 0.00023829	b = 6.28318531
i = 9284	x = 2.17864294	y = 0.21164124	h = 0.00023829	b = 6.28318531
i = 9285	x = 2.17888123	y = 0.2116

i = 9400	x = 2.20629280	y = 0.21428564	h = 0.00023843	b = 6.28318531
i = 9401	x = 2.20653123	y = 0.21430844	h = 0.00023843	b = 6.28318531
i = 9402	x = 2.20676966	y = 0.21433123	h = 0.00023843	b = 6.28318531
i = 9403	x = 2.20700810	y = 0.21435403	h = 0.00023843	b = 6.28318531
i = 9404	x = 2.20724653	y = 0.21437682	h = 0.00023844	b = 6.28318531
i = 9405	x = 2.20748497	y = 0.21439962	h = 0.00023844	b = 6.28318531
i = 9406	x = 2.20772341	y = 0.21442242	h = 0.00023844	b = 6.28318531
i = 9407	x = 2.20796185	y = 0.21444521	h = 0.00023844	b = 6.28318531
i = 9408	x = 2.20820029	y = 0.21446801	h = 0.00023844	b = 6.28318531
i = 9409	x = 2.20843873	y = 0.21449081	h = 0.00023844	b = 6.28318531
i = 9410	x = 2.20867717	y = 0.21451360	h = 0.00023844	b = 6.28318531
i = 9411	x = 2.20891562	y = 0.21453640	h = 0.00023844	b = 6.28318531
i = 9412	x = 2.20915406	y = 0.21455920	h = 0.00023845	b = 6.28318531
i = 9413	x = 2.20939251	y = 0.21458199	h = 0.00023845	b = 6.28318531
i = 9414	x = 2.20963095	y = 0.2146

i = 9586	x = 2.25066229	y = 0.21852579	h = 0.00023866	b = 6.28318531
i = 9587	x = 2.25090095	y = 0.21854858	h = 0.00023866	b = 6.28318531
i = 9588	x = 2.25113961	y = 0.21857138	h = 0.00023866	b = 6.28318531
i = 9589	x = 2.25137828	y = 0.21859418	h = 0.00023866	b = 6.28318531
i = 9590	x = 2.25161694	y = 0.21861697	h = 0.00023867	b = 6.28318531
i = 9591	x = 2.25185561	y = 0.21863977	h = 0.00023867	b = 6.28318531
i = 9592	x = 2.25209428	y = 0.21866257	h = 0.00023867	b = 6.28318531
i = 9593	x = 2.25233295	y = 0.21868536	h = 0.00023867	b = 6.28318531
i = 9594	x = 2.25257162	y = 0.21870816	h = 0.00023867	b = 6.28318531
i = 9595	x = 2.25281029	y = 0.21873096	h = 0.00023867	b = 6.28318531
i = 9596	x = 2.25304896	y = 0.21875375	h = 0.00023867	b = 6.28318531
i = 9597	x = 2.25328763	y = 0.21877655	h = 0.00023867	b = 6.28318531
i = 9598	x = 2.25352631	y = 0.21879935	h = 0.00023868	b = 6.28318531
i = 9599	x = 2.25376499	y = 0.21882214	h = 0.00023868	b = 6.28318531
i = 9600	x = 2.25400366	y = 0.2188

i = 9819	x = 2.30630447	y = 0.22383737	h = 0.00023896	b = 6.28318531
i = 9820	x = 2.30654343	y = 0.22386017	h = 0.00023896	b = 6.28318531
i = 9821	x = 2.30678239	y = 0.22388296	h = 0.00023896	b = 6.28318531
i = 9822	x = 2.30702135	y = 0.22390576	h = 0.00023896	b = 6.28318531
i = 9823	x = 2.30726031	y = 0.22392856	h = 0.00023896	b = 6.28318531
i = 9824	x = 2.30749927	y = 0.22395135	h = 0.00023896	b = 6.28318531
i = 9825	x = 2.30773823	y = 0.22397415	h = 0.00023896	b = 6.28318531
i = 9826	x = 2.30797719	y = 0.22399695	h = 0.00023896	b = 6.28318531
i = 9827	x = 2.30821616	y = 0.22401974	h = 0.00023897	b = 6.28318531
i = 9828	x = 2.30845513	y = 0.22404254	h = 0.00023897	b = 6.28318531
i = 9829	x = 2.30869410	y = 0.22406534	h = 0.00023897	b = 6.28318531
i = 9830	x = 2.30893306	y = 0.22408813	h = 0.00023897	b = 6.28318531
i = 9831	x = 2.30917203	y = 0.22411093	h = 0.00023897	b = 6.28318531
i = 9832	x = 2.30941101	y = 0.22413373	h = 0.00023897	b = 6.28318531
i = 9833	x = 2.30964998	y = 0.2241

StopIteration: ('Iteration number = ', 10000)

In [ ]:
plt.plot(x,y[:,1]-np.sin(x), label="y(x) Error")
plt.plot(x,y[:,0]-np.cos(x), label="z(x) Error")
plt.legend()